In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier #decision trees
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import hamming_loss
from sklearn.preprocessing import LabelEncoder
from sklearn import tree

In [2]:
#step 1: load data
enviro_data = pd.read_csv('data_participant.csv')
enviro_data

/var/folders/dx/66dwwfp55d1032n4tz8r0mz00000gp/T/ipykernel_12435/3796276621.py:2: DtypeWarning: Columns (1270,1271,1272) have mixed types. Specify dtype option on import or set low_memory=False.
  enviro_data = pd.read_csv('data_participant.csv')


,ID,year,month,day,hour,start_time,latitude,longitude,event,r_300.t-6,...,LD.t-2,LD.t-1,LD.t+0,LD.t+1,LD.t+2,LD.t+3,hail_size,y_thunderstorm,y_hail,y_severe
0,0,2005,4,8,4,4/8/2005 4:00,56.12900,-119.081980,NaN,98.769104,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False,False,False
1,1,2005,4,8,8,4/8/2005 8:00,52.12900,-119.081980,NaN,100.736720,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False,False,False
2,2,2005,4,13,2,4/13/2005 2:00,48.37900,-114.331790,NaN,33.332287,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False,False,False
3,3,2005,4,21,2,4/21/2005 2:00,53.87900,-105.831436,NaN,83.939270,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False,False,False
4,4,2005,5,3,21,5/3/2005 21:00,52.87900,-115.831850,NaN,95.186860,...,0.000048,0.000451,0.001993,0.004847,0.000247,0.001017,0.0,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9542,2022,6,30,1,6/30/2022 1:25,51.79841,-100.486810,2733.0,87.580380,...,2.484526,2.484526,2.484526,2.484526,2.484526,2.484526,21.0,NaN,NaN,NaN
9996,9543,2022,6,30,2,6/30/2022 2:20,51.89355,-99.637400,2733.0,86.444176,...,2.009431,2.009431,2.009431,2.009431,2.009431,2.009431,21.0,NaN,NaN,NaN
9997,9544,2022,6,30,2,6/30/2022 2:17,51.64718,-99.924870,2733.0,74.155975,...,1.972132,1.972132,1.972132,1.972132,1.972132,1.972132,21.0,NaN,NaN,NaN
9998,9545,2022,6,30,18,6/30/2022 18:55,55.74510,-97.850920,2734.0,15.329826,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.0,NaN,NaN,NaN


In [3]:
# step 2: clean data
df = enviro_data.drop(columns = ['year', 'month', 'day', 'hour', 'start_time', 'latitude', 'longitude', 'y_hail', 'y_severe'])
#remove event
df = df.drop(columns=['event'])
df

,ID,r_300.t-6,t_300.t-6,u_300.t-6,v_300.t-6,z_300.t-6,r_350.t-6,t_350.t-6,u_350.t-6,v_350.t-6,...,LD.t-4,LD.t-3,LD.t-2,LD.t-1,LD.t+0,LD.t+1,LD.t+2,LD.t+3,hail_size,y_thunderstorm
0,0,98.769104,224.61270,13.263535,36.174423,88575.600,106.095780,233.02351,13.736909,29.009111,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False
1,1,100.736720,223.20950,7.261487,39.146760,88316.766,100.252560,232.65732,2.325344,34.825794,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False
2,2,33.332287,223.15912,28.210712,32.365406,88134.125,37.167343,228.89455,29.782637,33.807636,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False
3,3,83.939270,225.64915,13.092142,-2.760100,90788.350,76.288240,234.56110,12.398405,-5.533770,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False
4,4,95.186860,220.30736,19.417390,1.879874,88811.770,87.919810,228.46283,15.501450,3.358401,...,0.000000,0.000000,0.000048,0.000451,0.001993,0.004847,0.000247,0.001017,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9542,87.580380,231.23580,27.343943,10.782755,91802.086,112.730070,239.94821,26.183979,10.610769,...,2.484526,2.484526,2.484526,2.484526,2.484526,2.484526,2.484526,2.484526,21.0,NaN
9996,9543,86.444176,231.15242,28.023106,10.422516,91733.336,104.506810,239.94946,27.061499,10.759514,...,2.009431,2.009431,2.009431,2.009431,2.009431,2.009431,2.009431,2.009431,21.0,NaN
9997,9544,74.155975,231.35771,29.122700,10.099461,91799.700,94.864586,239.91835,27.561707,11.324278,...,1.972132,1.972132,1.972132,1.972132,1.972132,1.972132,1.972132,1.972132,21.0,NaN
9998,9545,15.329826,230.93718,32.177850,4.024188,87779.484,21.187279,233.77641,34.521492,3.798748,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.0,NaN


In [4]:
#find prediction data
prediction_df = df[df['y_thunderstorm'].isna()]


#find remaining dataset
dataset = df.dropna(how='any')

#split into training and testing
X = dataset.drop(columns = ['y_thunderstorm'])
y = dataset['y_thunderstorm']

# convert true false to 0, 1
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
prediction_df

,ID,r_300.t-6,t_300.t-6,u_300.t-6,v_300.t-6,z_300.t-6,r_350.t-6,t_350.t-6,u_350.t-6,v_350.t-6,...,LD.t-4,LD.t-3,LD.t-2,LD.t-1,LD.t+0,LD.t+1,LD.t+2,LD.t+3,hail_size,y_thunderstorm
9062,3542,45.644350,234.42340,23.146078,-2.455483,92740.430,54.023254,243.10223,18.269053,-0.324257,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0.0,NaN
9063,3543,97.149980,225.17722,2.089920,-17.553024,89601.040,86.322250,234.23677,0.591454,-13.443966,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN
9064,3544,38.367930,229.15550,25.575968,-1.156295,91463.090,33.758232,237.79204,22.313837,-3.059757,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN
9065,3545,38.458720,238.12921,30.675068,-9.344202,94229.470,19.200207,246.49450,26.589320,-10.015875,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000815,0.0,NaN
9066,3546,102.628690,229.00185,-8.520090,7.480168,91293.266,77.873310,237.34350,-6.124699,7.354665,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9542,87.580380,231.23580,27.343943,10.782755,91802.086,112.730070,239.94821,26.183979,10.610769,...,2.484526,2.484526,2.484526,2.484526,2.484526,2.484526,2.484526,2.484526,21.0,NaN
9996,9543,86.444176,231.15242,28.023106,10.422516,91733.336,104.506810,239.94946,27.061499,10.759514,...,2.009431,2.009431,2.009431,2.009431,2.009431,2.009431,2.009431,2.009431,21.0,NaN
9997,9544,74.155975,231.35771,29.122700,10.099461,91799.700,94.864586,239.91835,27.561707,11.324278,...,1.972132,1.972132,1.972132,1.972132,1.972132,1.972132,1.972132,1.972132,21.0,NaN
9998,9545,15.329826,230.93718,32.177850,4.024188,87779.484,21.187279,233.77641,34.521492,3.798748,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.0,NaN


In [5]:
# create linear model
model = DecisionTreeClassifier()
model.fit(X_train.drop(columns=['ID']), y_train)

DecisionTreeClassifier()

In [6]:
predictions = model.predict(X_test.drop(columns=['ID']))

accuracy_score(y_test, predictions)

1.0

In [7]:
#actual predictions
act_predictions = model.predict(prediction_df.drop(columns = ['y_thunderstorm', 'ID']))

#convert actual predictions back to True False
pred_tf = act_predictions.astype(bool)
pred_tf

array([False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False,  True,
       False, False,  True, False,  True, False, False,  True, False,
       False, False, False,  True, False, False,  True, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True,  True,  True, False, False, False, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
        True, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [8]:
prediction_df.shape

(938, 1263)

In [9]:
# save predictions to existing csv file
prediction_df.loc[:,'y_thunderstorm'] = pred_tf
prediction_df
to_add = prediction_df[['ID', 'y_thunderstorm']]
to_add


,ID,y_thunderstorm
9062,3542,False
9063,3543,False
9064,3544,False
9065,3545,False
9066,3546,False
...,...,...
9995,9542,True
9996,9543,True
9997,9544,True
9998,9545,False


In [19]:
#replace original data with results, using ID
enviro_data
#to_add

enviro_data2 = enviro_data.merge(to_add, on='ID', how='left', suffixes=('', '_y'))
enviro_data2
enviro_data2

# Rename the 'results_y' column to 'results'
enviro_data2.rename(columns={'y_thunderstorm': 'y_thunderstorm'}, inplace=True)

# Drop the original 'results' column
enviro_data2.drop(columns='y_thunderstorm', inplace=True)
enviro_data2


,ID,year,month,day,hour,start_time,latitude,longitude,event,r_300.t-6,...,LD.t-2,LD.t-1,LD.t+0,LD.t+1,LD.t+2,LD.t+3,hail_size,y_hail,y_severe,y_thunderstorm_y
0,0,2005,4,8,4,4/8/2005 4:00,56.12900,-119.081980,NaN,98.769104,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False,False,NaN
1,1,2005,4,8,8,4/8/2005 8:00,52.12900,-119.081980,NaN,100.736720,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False,False,NaN
2,2,2005,4,13,2,4/13/2005 2:00,48.37900,-114.331790,NaN,33.332287,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False,False,NaN
3,3,2005,4,21,2,4/21/2005 2:00,53.87900,-105.831436,NaN,83.939270,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,False,False,NaN
4,4,2005,5,3,21,5/3/2005 21:00,52.87900,-115.831850,NaN,95.186860,...,0.000048,0.000451,0.001993,0.004847,0.000247,0.001017,0.0,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9542,2022,6,30,1,6/30/2022 1:25,51.79841,-100.486810,2733.0,87.580380,...,2.484526,2.484526,2.484526,2.484526,2.484526,2.484526,21.0,NaN,NaN,True
9996,9543,2022,6,30,2,6/30/2022 2:20,51.89355,-99.637400,2733.0,86.444176,...,2.009431,2.009431,2.009431,2.009431,2.009431,2.009431,21.0,NaN,NaN,True
9997,9544,2022,6,30,2,6/30/2022 2:17,51.64718,-99.924870,2733.0,74.155975,...,1.972132,1.972132,1.972132,1.972132,1.972132,1.972132,21.0,NaN,NaN,True
9998,9545,2022,6,30,18,6/30/2022 18:55,55.74510,-97.850920,2734.0,15.329826,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.0,NaN,NaN,False
